In [ ]:
import pandas as pd
import geopandas as gpd
import os

Download the [OS Open Names dataset](https://osdatahub.os.uk/data/downloads/open/OpenNames) and extract the zip into this directory, it should produce a directory called `opname_csv_gb`.

In [5]:
with open("opname_csv_gb/Doc/OS_Open_Names_Header.csv") as f:
    header = f.readline().split(',')

results = pd.DataFrame()
with os.scandir("opname_csv_gb/Data") as it:
    for entry in it:
        if entry.is_file() and entry.name.endswith(".csv"):
            df = pd.read_csv(entry.path, header=None, names=header)
            subset = df[df['TYPE'] == "populatedPlace"]
            results = pd.concat([results, subset[['NAME1', 'NAME2', 'GEOMETRY_X', 'GEOMETRY_Y']]])

/tmp/ipykernel_1100/3634243814.py:8: DtypeWarning: Columns (4,16,17,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(entry.path, header=None, names=header)
/tmp/ipykernel_1100/3634243814.py:8: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(entry.path, header=None, names=header)
/tmp/ipykernel_1100/3634243814.py:8: DtypeWarning: Columns (4,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(entry.path, header=None, names=header)
/tmp/ipykernel_1100/3634243814.py:8: DtypeWarning: Columns (4,31,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(entry.path, header=None, names=header)
/tmp/ipykernel_1100/3634243814.py:8: DtypeWarning: Columns (4,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(entry.path, header=None, nam

In [6]:
gdf = gpd.GeoDataFrame(results, geometry=gpd.points_from_xy(results['GEOMETRY_X'], results['GEOMETRY_Y'], crs=27700))

In [7]:
gdf = gdf.to_crs(4326)

In [8]:
gdf

,NAME1,NAME2,GEOMETRY_X,GEOMETRY_Y,geometry
0,Church Fenton,NaN,451388,436925,POINT (-1.22079 53.82606)
1,Kellingley,NaN,453010,424128,POINT (-1.19835 53.71089)
2,Half Acres,NaN,442612,425013,POINT (-1.35575 53.71979)
3,Newton,NaN,444690,427852,POINT (-1.32385 53.74514)
4,Chapel Haddlesey,NaN,458289,426091,POINT (-1.11801 53.72797)
...,...,...,...,...,...
12,West Yell,NaN,445549,1183102,POINT (-1.17194 60.52893)
13,Hamnavoe,NaN,449675,1180559,POINT (-1.09741 60.50562)
14,Colvister,NaN,451579,1197025,POINT (-1.05846 60.6532)
15,Aywick,NaN,453140,1186685,POINT (-1.03269 60.56017)


In [13]:
gdf['lat'] = round(gdf.geometry.y, 3)
gdf['lon'] = round(gdf.geometry.x, 3)

In [34]:
df = pd.read_csv('https://admin.opendatani.gov.uk/dataset/d27903f1-15e6-4c07-8564-ddc655e9c549/resource/411c6ea5-c079-43aa-bd62-438423d070cc/download/osni_open_data_-_gazetteer_-_place_names.csv')
igdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['X'], df['Y'], crs=29903))
igdf = igdf.to_crs(4326)
igdf['lat'] = round(igdf.geometry.y, 3)
igdf['lon'] = round(igdf.geometry.x, 3)
igdf

,PLACENAME,X,Y,OBJECTID,geometry,lat,lon
0,ACTON,305600,341200,1,POINT (-6.37836 54.30866),54.309,-6.378
1,AGHADOWEY,285900,420900,2,POINT (-6.65751 55.0281),55.028,-6.658
2,AGHALEE,312300,365000,3,POINT (-6.26648 54.52094),54.521,-6.266
3,AGIVEY,289900,422900,4,POINT (-6.59435 55.04535),55.045,-6.594
4,AHOGHILL,305000,401700,5,POINT (-6.36597 54.85205),54.852,-6.366
...,...,...,...,...,...,...,...
331,WARINGSFORD,324000,348900,332,POINT (-6.09263 54.37366),54.374,-6.093
332,WARINGSTOWN,310400,355300,333,POINT (-6.29942 54.43426),54.434,-6.299
333,WARRENPOINT,314300,318300,334,POINT (-6.25346 54.10116),54.101,-6.253
334,WHITEABBEY,336000,383000,335,POINT (-5.89239 54.67673),54.677,-5.892


In [35]:
output = pd.concat([
    gdf[['NAME1', 'lat', 'lon']], 
    gdf[['NAME2', 'lat', 'lon']].dropna().rename(columns={'NAME2': 'NAME1'}),
    igdf[['PLACENAME', 'lat', 'lon']].rename(columns={'PLACENAME': 'NAME1'})
    ]).to_json(orient='values')
with open("app/places.json", "w") as fd:
    fd.write(output)